In [1]:
import pandas as pd
import numpy as np
import copy
import json
from sklearn.model_selection import StratifiedKFold

from generateDescriptors import generateDescriptors
from sisso_classify import SissoClassifier
from Run_Sisso_Experiments import Run_Sisso_Experiment
from Run_Sisso_Experiments import get_no_dft_data

In [2]:
### Input Features

### Contains 190 compositions with binary experimental ordering types (rocksalt / disordered) and 
### reported order parameters (>0.0 is considered rock-salt)

### Candidate Features for model building include those related to 
### atomic properties: such as (electronegativity, oxidation state, and electronegativity)
### dft properties: such as (Probability of Rock-Salt, Energy difference btw Rock-Salt and Layered, Configurational S)

### Note: Descriptors in this set are unitless (e.g. ionic radii are divided by ionic radius of Oxygen)

input_data = pd.read_json("../data/perovskite_ordering_data.json")
input_data

,formula,A,B,A_electronegativity,A_ox_state,A_ionic_radius,B_electronegativity_diff,B_electronegativity_multiply,B_electronegativity_average,B_ox_state_diff,...,dft_rocksalt_prob,dft_normalized_conf_entropy,dft_rocksalt_layered_diff,min_dft_e_hull,max_unrelaxed_opt_cosine_distance,max_O_p_band_center,bandgap,O_p_band_center,exp_ordering_type,exp_ordering_parameter
0,Ba8Bi4O24Sb4,[Ba],"[Bi, Sb]",0.89,2,1.150000,0.15,3.8950,1.975,2,...,1.000000,0.163152,36.242399,0.006135,0.323957,-2.083131,"[0.0, 0.0, 0.0, 0.0, 0.0100025006, 1.3903475869]","[-2.8233435725, -2.2457582617, -2.749698896, -...",rs,0.98
1,Ba8Co4Mo4O24,[Ba],"[Co, Mo]",0.89,2,1.150000,0.28,4.0608,2.020,4,...,0.999999,0.163155,30.643952,0.002002,0.073721,-2.080090,"[0.0, 0.0, 0.4101025256, 0.1000250063, 0.12003...","[-2.8217294322, -2.452821915, -2.9428501722, -...",rs,0.99
2,Ba8Co4Nb4O24,[Ba],"[Co, Nb]",0.89,2,1.150000,0.28,3.0080,1.740,2,...,0.006704,0.431883,-3.875681,0.082667,0.053800,-1.603831,"[0.0, 0.3700925231, 0.0, 0.0, 0.0, 1.2103025756]","[-2.1206903026, -2.2537737394, -2.2195934315, ...",rs,0.30
3,Ba8Co4O24Re4,[Ba],"[Co, Re]",0.89,2,1.150000,0.02,3.5720,1.890,4,...,0.999838,0.163618,30.431283,0.062350,0.104040,-3.155874,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0900225056]","[-3.2445573329, -3.2727673463, -3.1558736936, ...",rs,0.90
4,Ba8Bi4Cr4O24,[Ba],"[Cr, Bi]",0.89,2,1.150000,0.24,3.1540,1.780,2,...,0.992378,0.178974,11.014691,0.108740,0.037398,-1.258229,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-1.6658440179, -1.5171371711, -1.7157137191, ...",d,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Cu4Mn4O24Tb8,[Tb],"[Cu, Mn]",1.10,3,0.782143,0.35,2.9450,1.725,2,...,0.000031,0.802066,-7.663441,0.328958,0.047919,-3.236550,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-3.5131533045000003, -3.5495137989, -3.236549...",d,0.00
186,Mn4Ni4O24Tb8,[Tb],"[Ni, Mn]",1.10,3,0.782143,0.36,2.9605,1.730,2,...,0.996473,0.169819,12.186483,0.292797,0.026247,-2.031711,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.5203800950000002]","[-3.2001235345, -3.3429092682, -3.3884557884, ...",rs,1.00
187,Mn4O24Sc4Tb8,[Tb],"[Sc, Mn]",1.10,3,0.782143,0.19,2.1080,1.455,0,...,0.008351,0.831453,-2.773813,0.053533,0.782871,-2.134919,"[0.9402350588, 0.6501625406, 0.6701675419, 0.4...","[-2.3443399911, -2.4301199423, -2.4050172107, ...",d,0.00
188,Cu4Mn4O24Tm8,[Tm],"[Cu, Mn]",1.25,3,0.751429,0.35,2.9450,1.725,2,...,0.000107,0.530221,-7.901864,0.439173,0.027307,-3.492907,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-3.6266842438999998, -3.7718519268, -3.549549...",d,0.00


In [3]:
### Constructing 

### Helper Function for parsing Input
initial_features, feature_names, experimental_labels = get_no_dft_data(input_data)

### Construct Candidate Descriptors for SISSO

[x_des,xVars_des,parents] = generateDescriptors(copy.deepcopy(initial_features),copy.deepcopy(feature_names),ops=2)

/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:105: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,i] = 1.0/X[:,i]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:125: RuntimeWarning: divide by zero encountered in log
  Xout[:,i] = np.log(X[:,i])
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: invalid value encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:45: RuntimeWarning: invalid value encountered in sqrt
  Xout[:,i] = X[:,i]**0.5
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:75: RuntimeWarning: invalid value encountered in power
  Xout[:,i] = X[

In [4]:
### Candidates is (num_data, num_descriptors)
print(x_des.shape)

### Example Name
ind = 245000
print(xVars_des[ind])

### Example of Parents - Parents[ind][0] and Parents[ind][1] contain parent features
print(xVars_des[parents[ind][0]])
print(xVars_des[parents[ind][1]])

(190, 498240)
(cos(X(B)X(B'))*(r(B_diff)*z(B_ave)))
cos(X(B)X(B'))
(r(B_diff)*z(B_ave))


In [5]:
### Running SISSO - Look at Just One Fold

skf =  StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

fold = 0
for i, (train_index, test_index) in enumerate(skf.split(experimental_labels,experimental_labels)):    
    if fold == 0:
        x_train = x_des[train_index]
        exp_train = experimental_labels[train_index]
        
        D = copy.deepcopy(x_train)
        P = copy.deepcopy(exp_train)
        
        classifier = SissoClassifier(1,1000,all_l0_combinations=False,weighted = False,SO_method = "domain_overlap")
        classifier.fit(D,P)
        
        selected_indices = classifier.l0_selected_indices[-1]
        best_1d_do = classifier.sis_selected_indices[0]
    
    fold+=1


/home/jypeng/miniconda3/envs/pvsk_order_des/lib/python3.9/site-packages/numpy/core/_methods.py:235: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [6]:
### selected_indices - Candidate descriptors selected by the SISSO algorithm

print(selected_indices)
print(xVars_des[selected_indices[0]])

### best_1d_do - Top 1000 Candidates that have the best domain overlap in the first SIS iteration

print(len(best_1d_do))
print(xVars_des[best_1d_do[435]])

[270738]
((r(B_ave)*r(B)r(B'))*(z(B_diff)+X(B_diff)))
1000
((z(B_diff)^(1/4))+(r(B_diff)*X(B_diff)))


In [7]:
### Example Running a Full Sisso Experiment and reading output file

In [8]:
Run_Sisso_Experiment("No_DFT",1,"domain_overlap",False)

Running Experiment 

Starting Fold 



/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:105: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,i] = 1.0/X[:,i]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:125: RuntimeWarning: divide by zero encountered in log
  Xout[:,i] = np.log(X[:,i])
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: invalid value encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:45: RuntimeWarning: invalid value encountered in sqrt
  Xout[:,i] = X[:,i]**0.5
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:75: RuntimeWarning: invalid value encountered in power
  Xout[:,i] = X[

Starting Fold 



/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:105: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,i] = 1.0/X[:,i]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:125: RuntimeWarning: divide by zero encountered in log
  Xout[:,i] = np.log(X[:,i])
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: invalid value encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:45: RuntimeWarning: invalid value encountered in sqrt
  Xout[:,i] = X[:,i]**0.5
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:75: RuntimeWarning: invalid value encountered in power
  Xout[:,i] = X[

Starting Fold 



...

Starting Fold 



/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:105: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,i] = 1.0/X[:,i]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:125: RuntimeWarning: divide by zero encountered in log
  Xout[:,i] = np.log(X[:,i])
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: invalid value encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:45: RuntimeWarning: invalid value encountered in sqrt
  Xout[:,i] = X[:,i]**0.5
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:75: RuntimeWarning: invalid value encountered in power
  Xout[:,i] = X[

Starting Fold 



/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:105: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,i] = 1.0/X[:,i]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:125: RuntimeWarning: divide by zero encountered in log
  Xout[:,i] = np.log(X[:,i])
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: divide by zero encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:192: RuntimeWarning: invalid value encountered in true_divide
  Xout[:,idxXout] = X1[:,i]/X2[:,j]
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:45: RuntimeWarning: invalid value encountered in sqrt
  Xout[:,i] = X[:,i]**0.5
/home/jypeng/repos/Perovskite-Ordering-Descriptors/sisso/generateDescriptors.py:75: RuntimeWarning: invalid value encountered in power
  Xout[:,i] = X[

Saving

../data/sisso_results/No_DFT_DescDim1_SoMethoddomain_overlap_.json



In [9]:
pd.read_json("../data/sisso_results/No_DFT_DescDim1_SoMethoddomain_overlap_.json")

,fpr,tpr,AUC
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714202,...","[0.0, 0.041666666666666005, 0.3333333333333330...",0.793155
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857101,...","[0.0, 0.041666666666666005, 0.25, 0.3333333333...",0.760417
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.041666666666666005, 0.0833333333333330...",0.940476
3,"[0.0, 0.0, 0.0, 0.07142857142857101, 0.0714285...","[0.0, 0.041666666666666005, 0.6666666666666661...",0.904762
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1333333333333...","[0.0, 0.043478260869565, 0.130434782608695, 0....",0.918841
